In [1]:
!pip install mediapipe
!pip install cvzone

In [1]:
import cv2
import numpy
import mediapipe as mp
import cvzone
import os
from cvzone.HandTrackingModule import HandDetector
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [2]:
class DragImg():
    def __init__(self, path, posOrigin, size, imgType):
 
        self.posOrigin = posOrigin
#         self.posNew = posOrigin
        self.imgType = imgType
        self.path = path
 
        if self.imgType == 'png':
            self.img = cv2.imread(self.path, cv2.IMREAD_UNCHANGED)
        else:
            self.img = cv2.imread(self.path)
 
        self.img = cv2.resize(self.img, (0,0),None,size,size)
 
        self.size = self.img.shape[:2]
 
    def update(self, cursor):
        ox, oy = self.posOrigin
        h, w = self.size
 
        # Check if in region
        if ox < cursor[0] < ox + w and oy < cursor[1] < oy + h:
            self.posOrigin = cursor[0] - w // 2, cursor[1] - h // 2
#             self.posNew = cursor[0] - w // 2, cursor[1] - h // 2

In [3]:
path = 'images'
paths = os.listdir(path)

In [4]:
paths

['dog.png', 'sunset_bg.jpeg', '.DS_Store', 'sandwich.png']

In [5]:
images = []
backgrounds = []
for x, img_path in enumerate(paths):
    if 'png' in img_path:
        img_type = 'png'
    else:
        img_type = 'jpg'
    if 'bg' in img_path:
        backgrounds.append(DragImg(f'{path}/{img_path}', [50 + x * 300, 50], 0.6,img_type))
    else:
        images.append(DragImg(f'{path}/{img_path}', [50 + x * 300, 50], 0.2,img_type))

error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [6]:
print(images)
print(backgrounds)

In [8]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
 
detector = HandDetector(detectionCon=0.8)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    rect1 = cv2.rectangle(img, (1200,400), (900, 700), (255, 0, 0), 2)
    rect2 = cv2.rectangle(img, (1200,50), (900, 350), (0, 255, 0), 2)
#     cv2.imshow('box1',img[50:350,900:1200])
    
    if hands:
        lmList = hands[0]['lmList']
        print(lmList)
        print(lmList[8][:2])
        print(lmList[12][:2])
        # Check if clicked
        length, info, img = detector.findDistance(lmList[8][:2], lmList[12][:2], img)
        print(length)
        if length < 60:
            cursor = lmList[8]
            for img_obj in images:
                img_obj.update(cursor)
            for img_obj in backgrounds:
                img_obj.update(cursor)
    
    try:
        for img_obj, bg_obj in images, backgrounds:

            # Draw for JPG image
            h, w = img_obj.size
            h_bg, w_bg = bg_obj.size
            
            ox, oy = img_obj.posOrigin
            ox_bg, oy_bg = bg_obj.posOrigin
            
            print('Position:',img_obj, (ox,oy))
            print('BG Position:',bg_obj, (ox_bg,oy_bg))
            if img_obj.imgType == "png":
                # Draw for PNG Images
                img = cvzone.overlayPNG(img, img_obj.img, [ox, oy])
            else:
                img[oy:oy + h, ox:ox + w] = img_obj.img
            if bg_obj.imgType == "png":
                # Draw for PNG Images
                img = cvzone.overlayPNG(img, bg_obj.img, [ox_bg, oy_bg])
            else:
                img[oy_bg:oy_bg + h_bg, ox_bg:ox_bg + w_bg] = bg_obj.img

        if (900,50) <= (ox_bg,oy_bg) <= (1200,350) and (900,400) <= (ox,oy) <= (1200,700) :
            print('ox_bg,oy_bg', ox_bg,oy_bg)
            print('ox,oy',ox,oy)
        
    except:
        pass

    cv2.imshow("Image", img)
#     cv2.imshow('box1', box1)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

	05-28-2022  20:49:57.757 @@@@===RelayStream:: set kCMIOStreamPropertyFormatDescription sub=2vuy w=1280 h=720
	05-28-2022  20:49:57.273 Local Camera start, ret=0
	05-28-2022  20:49:57.051 @@@@===RelayStream:: set kCMIOStreamPropertyFormatDescription sub=2vuy w=1280 h=720
	05-28-2022  20:49:57.330 Local Camera start, ret=0
Position: <__main__.DragImg object at 0x7fa2e9817670> (50, 50)
BG Position: <__main__.DragImg object at 0x7fa2e98171c0> (959, 458)
Position: <__main__.DragImg object at 0x7fa2e8e5b040> (927, 148)
BG Position: <__main__.DragImg object at 0x7fa2e9817e80> (734, 184)
